# Creating Fast API endpoint

In [1]:
!pip install fastapi uvicorn wandb pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.9 MB/s eta 0:00:00


In [2]:
#!pip uninstall pyngrok

In [2]:
!ngrok authtoken "<use key here>"

In [ ]:
import os

os.environ["WANDB_API_KEY"] = "<Use key here>"

In [5]:
%%writefile app.py

import os
import wandb
import sklearn
from joblib import load
from fastapi import FastAPI, HTTPException
from contextlib import asynccontextmanager
from pydantic import BaseModel
import wandb
import joblib
import pandas as pd
import numpy as np

app = FastAPI()

# Define input data schema
class PredictionRequest(BaseModel):
    Number_of_Customers: int
    Menu_Price: float
    Marketing_Spend: float
    Cuisine_Type: str
    Average_Customer_Spending: float
    Promotions: int
    Reviews: int


# Load the model from WandB
def load_model_from_wandb(project_name: str, artifact_name: str):
    try:
        # Initialize WandB
        run = wandb.init(project=project_name, job_type="inference", reinit=True)
        artifact = run.use_artifact(artifact_name)
        model_path = artifact.file()  # Assumes a single model file in the artifact
        model = joblib.load(model_path)
        run.finish()  # End the WandB run
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to load model: {e}")

project_name = "mlops_restautant_revenue"  # loading project name
artifact_name = "Linear_Model_restaurantrevenue:latest"  # loading artifact name
ml_model = load_model_from_wandb(project_name, artifact_name)

# Prediction endpoint
@app.post("/predict")
def predict(input_data: PredictionRequest):
    try:
        # Convert input data to a dictionary for prediction
        input_dict = input_data.dict()

        df = pd.DataFrame(input_dict, index = [0])

        # Call the model's prediction method
        prediction = ml_model.predict(df)

        # Return the prediction result
        return {f"Estimated restaurant revenue: DOLLAR {np.round(prediction[0], 2)}"}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {e}")


Writing app.py


In [6]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 &

nohup: appending output to 'nohup.out'


In [15]:
!tail nohup.out

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ritikranjan (ritikranjan-indian-school-of-business). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /content/wandb/run-20241212_173251-s0mmc6b9
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run confused-night-77
wandb: ⭐️ View project at https://wandb.ai/ritikranjan-indian-school-of-business/mlops_restautant_revenue
wandb: 🚀 View run at https://wandb.ai/ritikranjan-indian-school-of-business/mlops_restautant_revenue/runs/s0mmc6b9


In [22]:
!ps -ax | grep uvicorn

    365 ?        Sl     0:08 /usr/bin/python3 /usr/local/bin/uvicorn app:app --host 0.0.0.0 --port 8
    983 ?        S      0:00 /bin/bash -c ps -ax | grep uvicorn
    985 ?        S      0:00 grep uvicorn


In [18]:
from pyngrok import ngrok

# Expose the FastAPI app
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://a337-35-196-195-142.ngrok-free.app" -> "http://localhost:8000"


# Invoking the Prediction Service

In [19]:
import requests, json

data = """{ "Number_of_Customers": 81,
            "Menu_Price": 43.11763549,
            "Marketing_Spend": 12.66379252,
            "Cuisine_Type": "Japanese",
            "Average_Customer_Spending": 36.23613252,
            "Promotions": 0,
            "Reviews": 145
          }
         """
# Remove newline and extra spaces to convert the JSON data into a single-line string
json_data = data.replace('\n', '').replace(' ','')
json_data

'{"Number_of_Customers":81,"Menu_Price":43.11763549,"Marketing_Spend":12.66379252,"Cuisine_Type":"Japanese","Average_Customer_Spending":36.23613252,"Promotions":0,"Reviews":145}'

In [20]:
#
response = requests.post('https://a337-35-196-195-142.ngrok-free.app/predict', data=json_data)


In [21]:
print(response.json())

['Estimated restaurant revenue: DOLLAR 391.96']
